<a href="https://colab.research.google.com/github/AnetaGancheva/Notebooks/blob/main/MultiAgentSystem_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Calculate the distance to the furthest Irish pub in the world

A Multi Agent System practice:

Current scope: System of agents will calculate the travel distance between current position and the furthest Irish pub in the world.

In [1]:
!pip install 'smolagents[litellm]' matplotlib geopandas shapely kaleido -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
import math
from typing import Tuple
from smolagents import tool

@tool
def calculate_distance(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
) -> float:

    """
    A function to calculate the distance between two points on Earth based on their coordinates.

    Args:

    origin_coords: Tuple of (latitude, longitude) for starting point
    destination_coords: Tuple of (latitude, longitude) for end point

    Returns:
    float: the calculated distance in km between the two points

    Example:
        >>> # Varna, Bulgaria (43.2141° N, 27.9147° E) to Santiago, Chille (33.4489° S, 70.6693° W)
        >>> result = calculate_distance((43.2141, 27.9147), (-33.4489, -70.6693))
    """

    # Convert degrees to radians function
    def to_rads(degrees: float) -> float:
      return (degrees*(math.pi/180.0))

    # Extract coordinates
    lat_orig, lon_orig = map(to_rads, origin_coords)
    lat_dest, lon_dest = map(to_rads, destination_coords)

    # Earth's radius (km)
    EARTH_RADIUS_KM = 6371.0

    # Apply Haversine formula to get distance
    dlat = lat_dest - lat_orig
    dlon = lon_dest - lon_orig

    a = (
        math.sin(dlat/2) ** 2 + math.cos(lat_orig) * math.cos(lat_dest) * math.sin(dlon/2) ** 2
    )

    c = 2 * math.asin(math.sqrt(a))

    distance = EARTH_RADIUS_KM * c

    return distance


In [4]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, VisitWebpageTool

model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

In [5]:
web_agent = CodeAgent(
    model= model,
    tools=[ DuckDuckGoSearchTool(), VisitWebpageTool(), calculate_distance ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

In [6]:
import os
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel
from PIL import Image


def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True

In [7]:
manager_agent = CodeAgent(
    model=HfApiModel("deepseek-ai/DeepSeek-R1", provider="together"),
    tools=[calculate_distance],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15
)

In [8]:
manager_agent.run("""
Find the locations of all Irish pubs in the world and calculate the distance to Varna, Bulgaria (43.2141° N, 27.9147° E).
Represent this as a spatial map of the world, with the locations represented as scatter ploints with a color that depends on travel time.

Here is an example of how to plot and return a map:

df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you will see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find the locations of all Irish pubs in the world and calculate the distance to Varna, Bulgaria (43.2141° N,    │
│ 27.9147° E).                                                                                                    │
│ Represent this as a spatial map of the world, with the locations represented as scatter ploints with a color    │
│ that depends on travel time.                                                                                    │
│                                                                                                                 │
│ Here is an example of how to plot and return a map:                                                             │
│                                                                                                                 │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you will see it.     │
│                                                                                                                 │
╰─ HfApiModel - deepseek-ai/DeepSeek-R1 ──────────────────────────────────────────────────────────────────────────╯

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
```
<think>
Okay, the user sent "[object Object]". Hmm, that looks like a JavaScript object that's been converted to a string 
accidentally. Maybe they tried to send an object but it didn't stringify properly. I should let them know about the
error and ask for more details. I need to be friendly and helpful. Perhaps suggest checking their code or providing
more context so I can assist better.

Wait, maybe they're working with JSON and encountered this issue. The [object Object] usually happens when you do 
something like alert(obj) instead of JSON.stringify(obj). So, the user might not realize their data isn't showing 
up correctly. I should explain that and ask them to clarify what they need help with. Make sure to keep the 
response clear and approachable.
</think>

It looks like there was an issue with the data you sent—it appears as "[object Object]", which typically happens 
when a JavaScript object is converted to a string unintentionally. Could you clarify or provide more details about 
what you need help with? For example, are you working with code, debugging an error, or asking a specific question?
Let me know so I can assist! 😊
```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
[RESPONSE]:                                                                                                        
It looks like there was an issue with the data you sent—it appears as "[object Object]", which typically happens   
when a JavaScript object is converted to a string unintentionally. Could you clarify or provide more details about 
what you need help with? For example, are you working with code, debugging an error, or asking a specific question?
Let me know so I can assist! 😊                                                                                    

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: https://huggingface.co/api/inference-proxy/together/v1/chat/completions
(Request ID: Root=1-67cc3a8a-7a2e33341a8df8a410b8d7b7;5284b0a5-85ca-4c2e-a2ca-858d92f6046a)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
allowance.

[Step 5: Duration 0.17 seconds| Input tokens: 2,630 | Output tokens: 425]

[Step 6: Duration 0.05 seconds| Input tokens: 3,156 | Output tokens: 510]

HfHubHTTPError: 402 Client Error: Payment Required for url: https://huggingface.co/api/inference-proxy/together/v1/chat/completions (Request ID: Root=1-67cc3a8a-60a9e6cb024fa62c1950f1c9;7c09588c-8f39-4648-940e-b31ac90dd920)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly allowance.